In [199]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import lbl2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from pyspark.sql.functions import date_format
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


In [200]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/10/04 01:22:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [201]:
# Read in data from ETL.py file
%run '../scripts/ETL.py' '../scripts/paths.json'
final_join3.limit(5)

22/10/04 01:22:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021
Egestas Nunc Asso...,11121775571,digital goods: bo...,6.58,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,11121775571,11.28829564583802,2bd2a61d-72e5-42d...,2021-08-20,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Morbi Accumsan In...,19618998054,tent and aWning s...,1.52,c,Christopher Rodri...,30554 Evans Strea...,NSW,Male,19618998054,62.90176609196828,3582b1f8-4577-403...,2021-05-16,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Dolor Egestas PC,94472466107,"cable, satellite,...",6.23,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,94472466107,172.15375126873164,cb05d49f-c2fa-453...,2021-07-22,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Urna Justo Indust...,31472801314,music shops - mus...,6.56,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,31472801314,0.4894787650356477,aeec15c1-67e8-4cb...,2021-05-18,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Sem Pellentesq...,35424691626,"computers, comput...",3.9,b,Christopher Rodri...,30554 Evans Strea...,NSW,Male,35424691626,7.360217018778133,9df473ba-102d-461...,2021-07-04,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694


In [202]:
final_join3.count()

10540181

In [203]:
tagged_merchants = pd.read_csv("../data/curated/tagged_merchants.csv")
tagged_merchants = tagged_merchants.iloc[:,1:]
tagged_merchants.drop(['tags', 'name', 'cleaned_tags', 'store_type'], axis=1, inplace=True)
tagged_merchants.to_parquet("../data/curated/tagged_merchants.parquet")
tagged_merchants_sdf = spark.read.parquet("../data/curated/tagged_merchants.parquet")

In [204]:
tagged_merchants_sdf = tagged_merchants_sdf.withColumnRenamed('merchant_abn',

    'tagged_merchant_abn'
)

In [205]:
tagged_merchants_sdf.show(5)

+-------------------+--------------------+
|tagged_merchant_abn|            category|
+-------------------+--------------------+
|        10023283211|           Furniture|
|        10142254217|         Electronics|
|        10165489824|        Toys and DIY|
|        10187291046|        Toys and DIY|
|        10192359162|Books, Stationary...|
+-------------------+--------------------+
only showing top 5 rows



In [206]:
final_join3.createOrReplaceTempView("join")
tagged_merchants_sdf.createOrReplaceTempView("tagged")

joint = spark.sql(""" 

SELECT *
FROM join
INNER JOIN tagged
ON join.merchant_abn = tagged.tagged_merchant_abn
""")

joint = joint.drop('tagged_merchant_abn')

In [207]:
joint.count()

10109254

In [208]:
joint.createOrReplaceTempView("group")

a = spark.sql(""" 

SELECT *, (dollar_value - take_rate) AS total_earning
FROM group
""")

In [209]:
a.createOrReplaceTempView("group_earnings")

b = spark.sql(""" 

SELECT merchant_name, SUM(total_earning) AS total_revenue, COUNT(merchant_abn) AS no_of_transactions
FROM group_earnings
GROUP BY merchant_name
""")

b.show(5)

+--------------------+------------------+------------------+
|       merchant_name|     total_revenue|no_of_transactions|
+--------------------+------------------+------------------+
|   Dictum Mi Limited| 897687.9661494899|              9880|
|Volutpat Nulla In...| 836578.8864434501|             10561|
|Mollis Integer Co...|384905.39521080774|             12234|
|Hendrerit Consect...|239274.53116224083|              3010|
|Donec Luctus Indu...|  1063577.03533018|              3113|
+--------------------+------------------+------------------+
only showing top 5 rows



In [210]:
# Extracting the year, month, day from the timestamp

a = a.withColumn("Year", 
date_format('order_datetime', 'yyyy'))

a  = a.withColumn("Month", 
date_format('order_datetime', 'MMMM'))


a = a.withColumn("Day",
date_format(("order_datetime"), "E"))


In [217]:
a = a.drop('merchant_abn', 'categories','name', 'address', 'trans_merchant_abn', 'order_id','order_datetime','user_id','consumer_id','int_sa2','SA2_code',
'SA2_name','income_2018-2019','total_males', 'total_females','total_persons', 'state_code','state_name','population_2020', 'population_2021','total_earning')

In [218]:
 
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
train_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_data.columns]
   ).show()

+-------------+---------+--------------+-----+------+------------+---------+--------+----+-----+---+
|merchant_name|take_rate|revenue_levels|state|gender|dollar_value|postcodes|category|Year|Month|Day|
+-------------+---------+--------------+-----+------+------------+---------+--------+----+-----+---+
|            0|        0|             0|    0|     0|           0|        0|       0|   0|    0|  0|
+-------------+---------+--------------+-----+------+------------+---------+--------+----+-----+---+



In [219]:
a.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- postcodes: string (nullable = true)
 |-- category: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



In [ ]:

merchant name
take rate
revenue levels
number of transactions
year
month
sum(dollar value)
numer of males
number of females
category


aggregate merchant name by year, month



In [220]:
a.createOrReplaceTempView("agg")

male = spark.sql(""" 

SELECT merchant_name AS m_name, COUNT(gender) as males
FROM agg
WHERE gender = 'Male'
GROUP BY merchant_name
""")

male.show(5)

female = spark.sql(""" 

SELECT merchant_name AS f_name, COUNT(gender) as females
FROM agg
WHERE gender = 'Female'
GROUP BY merchant_name
""")
female.show(5)

+--------------------+-----+
|              m_name|males|
+--------------------+-----+
|Elit Sed Consequa...| 4376|
|Varius Orci Insti...| 8966|
|Mollis Integer Co...| 5507|
|Donec Luctus Indu...| 1367|
|Volutpat Nulla In...| 4730|
+--------------------+-----+
only showing top 5 rows



+--------------------+-------+
|              f_name|females|
+--------------------+-------+
|Varius Orci Insti...|   8743|
|Volutpat Nulla In...|   4711|
|Donec Luctus Indu...|   1437|
|Mollis Integer Co...|   5456|
|      Montes Limited|    310|
+--------------------+-------+
only showing top 5 rows



In [222]:
a.createOrReplaceTempView("agg")

temp = spark.sql(""" 

SELECT merchant_name, COUNT(merchant_name) AS no_of_transactions, Year, Month, Day, SUM(dollar_value) AS total_earnings
FROM agg
GROUP BY merchant_name, Year, Month, Day
""")

temp.show()


+--------------------+------------------+----+------+---+------------------+
|       merchant_name|no_of_transactions|Year| Month|Day|    total_earnings|
+--------------------+------------------+----+------+---+------------------+
|      Vestibulum LLP|                34|2021|August|Thu| 1641.724325853504|
|      Mattis Limited|                11|2021|August|Sat| 8898.058932754078|
|Maecenas Iaculis ...|                 4|2021|August|Sun|3042.5175241306893|
|Ipsum Dolor Sit C...|              1110|2021|  June|Wed| 41343.31412523744|
|Hendrerit Donec C...|                48|2021|  June|Wed|2796.5712834332867|
|Suspendisse Dui C...|              1029|2021|   May|Thu|30985.432710066725|
|Euismod Urna Inst...|               344|2021|  June|Fri|5079.2464805916625|
|   In Nec Industries|                65|2021|August|Sat| 5203.336031953083|
|          Ac Limited|                94|2021|August|Sat|2914.4391822851817|
|Aliquam Iaculis L...|                25|2021|August|Mon| 999.8642553610293|

In [223]:
temp.createOrReplaceTempView("gender_join")
male.createOrReplaceTempView("m")
female.createOrReplaceTempView("f")

temp2 = spark.sql(""" 

SELECT *
FROM gender_join
INNER JOIN m
ON gender_join.merchant_name = m.m_name
""")

temp2.createOrReplaceTempView("temp2")

temp3 = spark.sql(""" 

SELECT *
FROM temp2
INNER JOIN f
ON temp2.merchant_name = f.f_name
""")

temp3.limit(5)

merchant_name,no_of_transactions,Year,Month,Day,total_earnings,m_name,males,f_name,females
Donec Luctus Indu...,22,2021,June,Thu,7205.231203961974,Donec Luctus Indu...,1367,Donec Luctus Indu...,1437
Malesuada PC,2,2021,May,Tue,289.7467691470908,Malesuada PC,60,Malesuada PC,70
Volutpat Nulla In...,53,2021,September,Mon,3771.9649861122953,Volutpat Nulla In...,4730,Volutpat Nulla In...,4711
Porttitor Eros In...,11,2021,October,Tue,1302.8216868924953,Porttitor Eros In...,560,Porttitor Eros In...,579
Porttitor Eros In...,10,2022,October,Sat,954.8397782336599,Porttitor Eros In...,560,Porttitor Eros In...,579


In [224]:
a.createOrReplaceTempView("features")

e = spark.sql(""" 

SELECT merchant_name AS drop_name, FIRST(take_rate) AS take_rate, FIRST(revenue_levels) AS revenue_levels, FIRST(category) AS category
FROM features
GROUP BY merchant_name
""")

e.show(2)

+----------------+---------+--------------+--------------------+
|       drop_name|take_rate|revenue_levels|            category|
+----------------+---------+--------------+--------------------+
|    A Associates|     4.95|             b|Books, Stationary...|
|A Enim Institute|     6.49|             a|        Toys and DIY|
+----------------+---------+--------------+--------------------+
only showing top 2 rows



In [225]:
temp3.createOrReplaceTempView("edit")
e.createOrReplaceTempView("rates")

temp4 = spark.sql(""" 

SELECT *
FROM edit
INNER JOIN rates
ON edit.merchant_name = rates.drop_name
""")

train = temp4.drop('m_name', 'f_name', 'drop_name')

train.limit(5)

merchant_name,no_of_transactions,Year,Month,Day,total_earnings,males,females,take_rate,revenue_levels,category
Donec Luctus Indu...,22,2021,June,Thu,7205.231203961974,1367,1437,1.13,d,"Books, Stationary..."
Malesuada PC,2,2021,May,Tue,289.7467691470908,60,70,6.87,a,Furniture
Volutpat Nulla In...,53,2021,September,Mon,3771.9649861122953,4730,4711,6.59,a,Furniture
Porttitor Eros In...,11,2021,October,Tue,1302.8216868924953,560,579,3.86,b,Electronics
Porttitor Eros In...,10,2022,October,Sat,954.8397782336599,560,579,3.86,b,Electronics


In [ ]:
train.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- no_of_transactions: long (nullable = false)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- total_earnings: double (nullable = true)
 |-- males: long (nullable = false)
 |-- females: long (nullable = false)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)



In [226]:
# String indexing the categorical columns

indexer = StringIndexer(inputCols = ['merchant_name', 'Year', 'Month', 'Day', 'revenue_levels','category'],
outputCols = ['merchant_name_num', 'Year_num', 'Month_num', 'Day_num', 'revenue_levels_num','category_num'])

indexd_data = indexer.fit(train).transform(train)


# Applying onehot encoding to the categorical data that is string indexed above
encoder = OneHotEncoder(inputCols = ['merchant_name_num', 'Year_num', 'Month_num', 'Day_num', 'revenue_levels_num','category_num'],
outputCols = ['merchant_name_vec', 'Year_vec', 'Month_vec', 'Day_vec', 'revenue_levels_vec','category_vec'])

onehotdata = encoder.fit(indexd_data).transform(indexd_data)


# Assembling the training data as a vector of features 
assembler1 = VectorAssembler(
inputCols=['no_of_transactions','take_rate', 'merchant_name_vec', 'Year_vec', 'Month_vec', 'Day_vec', 'revenue_levels_vec','category_vec'],
outputCol= "features" )

outdata1 = assembler1.transform(onehotdata)

In [227]:
# Renaming the target column as label

outdata1 = outdata1.withColumnRenamed(
    "total_earnings",
    "label"
)

In [228]:
# Assembling the features as a feature vector 

featureIndexer =\
    VectorIndexer(inputCol="features", 
    outputCol="indexedFeatures").fit(outdata1)

outdata1 = featureIndexer.transform(outdata1)

In [229]:
# Split the data into training and validation sets (30% held out for testing)

trainingData, testData = outdata1.randomSplit([0.7, 0.3], seed = 20)

In [230]:
trainingData.count(), testData.count()

22/10/04 01:36:39 WARN DAGScheduler: Broadcasting large task binary with size 1653.6 KiB


22/10/04 01:37:08 WARN DAGScheduler: Broadcasting large task binary with size 1653.6 KiB


(276598, 118292)

In [231]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")


# Train model.  
model = rf.fit(trainingData)

# Make predictions.
predictions_validation = model.transform(testData)

22/10/04 01:37:34 WARN DAGScheduler: Broadcasting large task binary with size 1660.4 KiB


22/10/04 01:37:35 WARN DAGScheduler: Broadcasting large task binary with size 1660.5 KiB


22/10/04 01:37:38 WARN DAGScheduler: Broadcasting large task binary with size 1664.4 KiB


22/10/04 01:37:40 WARN DAGScheduler: Broadcasting large task binary with size 1878.5 KiB


22/10/04 01:37:55 WARN DAGScheduler: Broadcasting large task binary with size 1990.1 KiB


22/10/04 01:38:04 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


22/10/04 01:38:14 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


22/10/04 01:38:24 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


In [232]:
# Evaluate the validation set 

predictions_validation.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error

evaluator_train_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse_train = evaluator_train_rmse.evaluate(predictions_validation)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_train)

evaluator_train_mae = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae_train = evaluator_train_mae.evaluate(predictions_validation)
print("Root Mean Squared Error (MAE) on raint data = %g" % mae_train)

22/10/04 01:38:53 WARN DAGScheduler: Broadcasting large task binary with size 1667.9 KiB


+-----------------+------------------+--------------------+
|       prediction|             label|            features|
+-----------------+------------------+--------------------+
|2436.713456453721| 309.7048895088428|(4002,[0,1,1232,3...|
|2436.713456453721|1059.4983614151868|(4002,[0,1,1232,3...|
|2436.713456453721|262.07346569249825|(4002,[0,1,1232,3...|
|2436.713456453721| 679.4451079732337|(4002,[0,1,1232,3...|
|2819.627218939699|1503.2048370295236|(4002,[0,1,1232,3...|
+-----------------+------------------+--------------------+
only showing top 5 rows



22/10/04 01:39:16 WARN DAGScheduler: Broadcasting large task binary with size 1665.7 KiB


22/10/04 01:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1666.8 KiB
Root Mean Squared Error (RMSE) on train data = 5997.05


22/10/04 01:39:38 WARN DAGScheduler: Broadcasting large task binary with size 1665.7 KiB


22/10/04 01:39:40 WARN DAGScheduler: Broadcasting large task binary with size 1666.8 KiB
Root Mean Squared Error (MAE) on raint data = 3096.87
